### Hyperspy notebook to generate video from a series of TEM image
#### by Martial Duchamp NTU-MSE, Singapore m.duchamp@ntu.edu.sg

In [1]:
#from PIL import Image
import matplotlib.pyplot as plt
#import time
#import csv
#from os import listdir
#from os.path import isfile, join
#import pandas as pd
import matplotlib.animation as animation
import matplotlib as mpl
#from scipy import interpolate
from scipy import ndimage
#import numpy as np, cv2
#import matplotlib.patches as patches

mpl.rcParams['axes.linewidth'] = 3.0 #set the value globally
#mpl.use("Agg")

In [2]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [3]:
#import sys 
#sys.path.insert(0, '/home/martial/Data/programs/Hyperspy_ungit')
import hyperspy.api as hs

In [4]:
hs.hyperspy.Release.info

'\n    H y p e r S p y\n    Version 1.5.2\n\n    http://www.hyperspy.org\n\n    '

### Read TEM images

In [21]:
#path = '/mnt/nisbfs/Vaso/2020-12-28/TEM_time-resolved/Test-2'
i=3
TEM = hs.load('image_'+np.str(i)+',*.dm4')

In [22]:
TEM

<Signal2D, title: image_3, temperature_386.566, dimensions: (|4096, 4096)>

In [23]:
TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical

├── Applied Voltage(V) = 1.02
├── Measured Current(A) = 0.00554826
├── Resistance(Ohms)  = 183.8414205534708
└── Temperature(K)  = 386.56568221388324

In [9]:
R= TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV / \
    TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA

In [10]:
#R=bT+c
#R(400)=180=b*400+c  #image 99
#R(140)=97.5=b*140+c  #image10
#180-97.5=b(400-140)
b=(180-97.5)/(400-140)
b

0.3173076923076923

In [12]:
T = (R -c)/b
T

361.4618931226878

### Video

In [29]:
FFMpegWriter = animation.writers['ffmpeg']
metadata = dict(title='Test-2_b='+'{0:.0f}'.format(b)+'_c='+'{0:.0f}'.format(c), artist='M. Duchamp')
writer = FFMpegWriter(fps=0.5, metadata=metadata, bitrate=200)

In [30]:
vmin_mul = 0.
vmax_mul = 2.5

In [31]:
i=0
fig, axes = plt.subplots(nrows=1, ncols=1)
TEM = hs.load('image_'+np.str(i)+',*.dm4')
im = axes.imshow(ndimage.gaussian_filter(TEM.data,6), cmap='gray', \
                  vmin=vmin_mul*np.mean(TEM.data), vmax=vmax_mul*np.mean(TEM.data))

R= TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV / \
    TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA

#if R <0.001: R=b*130+c
T=TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.TemperatureK

textbox = plt.text(0.8, 0.8, \
    '{0:.0f}'.format(T) +'K '+str(i), \
    fontsize=15, verticalalignment='top', backgroundcolor='white') 

plt.show()

In [28]:
with writer.saving(fig, 'Test-3_fps03.mp4', dpi=300):
    for i in range(3,210):
        TEM = hs.load('image_'+np.str(i)+',*.dm4')
        im = axes.imshow(ndimage.gaussian_filter(TEM.data,6), cmap='gray', \
                         vmin=vmin_mul*np.mean(TEM.data), vmax=vmax_mul*np.mean(TEM.data))           

       # R= TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV / \
       #     TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA
        #if R <0.001: R=b*130+c
        T=TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.TemperatureK
        textbox = plt.text(0.8, 0.8, '{0:.0f}'.format(T) +'K '+str(i), \
            fontsize=15, verticalalignment='top', backgroundcolor='white')        
        
        writer.grab_frame()
        textbox.remove() 
        print(i)
        
    for j in range(0,3):
        im = axes.imshow(ndimage.gaussian_filter(TEM.data,6), cmap='gray', \
                         vmin=vmin_mul*np.mean(TEM.data), vmax=vmax_mul*np.mean(TEM.data))           

        #R= TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV / \
        #    TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA
       # if R <0.001: R=b*130+c
        T=TEM.original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.TemperatureK   
        textbox = plt.text(0.8, 0.8, '{0:.0f}'.format(T) +'K '+str(i), \
            fontsize=15, verticalalignment='top', backgroundcolor='white')  
        
        writer.grab_frame()
        textbox.remove() 
        print(i)

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
209
209
209


### Video crop center

In [ ]:
FFMpegWriter = animation.writers['ffmpeg']
metadata = dict(title='2020-12-28/TEM_time-resolved/Test-1 cropped', artist='M. Duchamp')
writer = FFMpegWriter(fps=1.0, metadata=metadata, bitrate=200)

In [ ]:
TEM[0].crop_image(top=30., left=30., bottom=70., right=70.)

In [ ]:
TEM[0].plot()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1)
TEM[0].crop_image(top=30., left=30., bottom=70., right=70.)
im = axes.imshow(ndimage.gaussian_filter(TEM[0].data,6), cmap='gray', \
                  vmin=0.3*np.mean(TEM[0].data), vmax=20*np.mean(TEM[0].data))

R= TEM[0].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV / \
    TEM[0].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA

if R <0.001: R=b*130+c

textbox = plt.text(0.8, 0.8, \
    '{0:.0f}'.format((R - c)/b) +'K', \
    fontsize=15, verticalalignment='top', backgroundcolor='white') 

plt.show()

In [ ]:
with writer.saving(fig, "diffraction-crop.mp4", dpi=300):
    for i in range(0,TEM.count(0)):
        if i > 0: TEM[i].crop_image(top=30., left=30., bottom=70., right=70.)
        im = axes.imshow(ndimage.gaussian_filter(TEM[i].data,6), cmap='gray', \
                         vmin=0.3*np.mean(TEM[i].data), vmax=20*np.mean(TEM[i].data))           

        R= TEM[i].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV / \
            TEM[i].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA
        if R <0.001: R=b*130+c
            
        textbox = plt.text(0.8, 0.8, '{0:.0f}'.format((R - c)/b) +'K', \
            fontsize=15, verticalalignment='top', backgroundcolor='white')        
        
        writer.grab_frame()
        textbox.remove() 
        print(i)
        
    for j in range(0,3):
        im = axes.imshow(ndimage.gaussian_filter(TEM[i].data,6), cmap='gray', \
                         vmin=0.3*np.mean(TEM[i].data), vmax=20*np.mean(TEM[i].data))           

        R= TEM[i].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Applied_VoltageV / \
            TEM[i].original_metadata.ImageList.TagGroup0.ImageTags.Default_tags.Electrical.Measured_CurrentA
        if R <0.001: R=b*130+c
            
        textbox = plt.text(0.8, 0.8, '{0:.0f}'.format((R - c)/b) +'K', \
            fontsize=15, verticalalignment='top', backgroundcolor='white')  
        
        writer.grab_frame()
        textbox.remove() 
        print(i)

In [ ]:
R

### Local functions not needed for this script

In [ ]:
def align2D_cv(hs_stack, warp_mode):
    
    hs_stack.change_dtype('float32')

    #shift
    warp_matrix_max = np.zeros(4)
    
    # High pass filter using low pass and Gaussian blurring
    filter_1 = ndimage.gaussian_filter(hs_stack.data[0], 10)
    
    # Convert Numpy hpf data in openCV format (3 channels)
    img_1_hpf_cv = cv2.cvtColor(filter_1, cv2.COLOR_GRAY2BGR)
    # Convert openCV format (3 channels) in openCV format grey (1 channel)
    img_1_hpf_cv = cv2.cvtColor(img_1_hpf_cv, cv2.COLOR_BGR2GRAY)
     
    #find the shape
    sz = img_1_hpf_cv.shape
    
    #Translation ( MOTION_TRANSLATION ) : The first image can be shifted ( translated ) by (x , y) to obtain the second image. There are only two parameters x and y that we need to estimate.
    #Euclidean ( MOTION_EUCLIDEAN ) : The first image is a rotated and shifted version of the second image. So there are three parameters — x, y and angle . You will notice in Figure 4, when a square undergoes Euclidean transformation, the size does not change, parallel lines remain parallel, and right angles remain unchanged after transformation.
    #Affine ( MOTION_AFFINE ) : An affine transform is a combination of rotation, translation ( shift ), scale, and shear. This transform has six parameters. When a square undergoes an Affine transformation, parallel lines remain parallel, but lines meeting at right angles no longer remain orthogonal.
    #Homography ( MOTION_HOMOGRAPHY ) : All the transforms described above are 2D transforms. They do not account for 3D effects. A homography transform on the other hand can account for some 3D effects ( but not all ). This transform has 8 parameters. A square when transformed using a Homography can change to any quadrilateral.
    
    #warp_mode = cv2.MOTION_TRANSLATION
    
    # Define 2x3 or 3x3 matrices and initialize the matrix to identity
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else :
        warp_matrix = np.eye(2, 3, dtype=np.float32)
        
    warp_matrix_returned = np.empty((hs_stack.data.shape[0], warp_matrix.shape[0], warp_matrix.shape[1]))
    
    # Specify the number of iterations.
    number_of_iterations = 3000;
    
    # Specify the threshold of the increment
    termination_eps = 1e-10;
    
    # Define termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)

    for i in range (1,hs_stack.data.shape[0]):
        filter_2 = ndimage.gaussian_filter(hs_stack.data[i], 10)
        # Convert Numpy hpf data in openCV format (3 channels)
        img_2_hpf_cv = cv2.cvtColor(filter_2, cv2.COLOR_GRAY2BGR)
        # Convert openCV format (3 channels) in openCV format grey (1 channel)
        img_2_hpf_cv = cv2.cvtColor(img_2_hpf_cv, cv2.COLOR_BGR2GRAY)
    
        # Run the ECC algorithm. The results are stored in warp_matrix.
        (cc, warp_matrix) = cv2.findTransformECC(img_1_hpf_cv,img_2_hpf_cv,warp_matrix, warp_mode, criteria)
    
        warp_matrix_returned[i,:,:] = warp_matrix
    
        if warp_mode == cv2.MOTION_HOMOGRAPHY :
            # Use warpPerspective for Homography 
            hs_stack.data[i] = cv2.warpPerspective (hs_stack.data[i], warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        else :
            # Use warpAffine for Translation, Euclidean and Affine
            hs_stack.data[i] = cv2.warpAffine(hs_stack.data[i], warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP); 
           
        if warp_matrix[0,2]<warp_matrix_max[0]:
            warp_matrix_max[0]=warp_matrix[0,2]
            
        if warp_matrix[0,2]>warp_matrix_max[1]:
            warp_matrix_max[1]=warp_matrix[0,2]

        if warp_matrix[1,2]<warp_matrix_max[2]:
            warp_matrix_max[2]=warp_matrix[1,2]
            
        if warp_matrix[1,2]>warp_matrix_max[3]:
            warp_matrix_max[3]=warp_matrix[1,2]
            
        print(str(i) +' / '+str(hs_stack.data.shape[0]-1))
            
    hs_stack.crop_image(bottom=hs_stack.data.shape[2]-np.int(warp_matrix_max[3]), \
                        top = -np.int(warp_matrix_max[2]), \
                        left=-np.int(warp_matrix_max[0]), \
                        right = hs_stack.data.shape[1]-np.int(warp_matrix_max[1]))
    
    return warp_matrix_returned

In [ ]:
def align2D_cv_only_align(hs_stack, warp_matrix, warp_mode):
    
    hs_stack.change_dtype('float32')

    #shift
    warp_matrix_max = np.zeros(4)
       
    #find the shape
    sz = hs_stack.data[0].shape
        
    for i in range (1,hs_stack.data.shape[0]):
        if warp_mode == cv2.MOTION_HOMOGRAPHY :
            # Use warpPerspective for Homography 
            hs_stack.data[i] = cv2.warpPerspective (hs_stack.data[i], warp_matrix[i,:,:], (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        else :
            # Use warpAffine for Translation, Euclidean and Affine
            hs_stack.data[i] = cv2.warpAffine(hs_stack.data[i], warp_matrix[i,:,:], (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP); 
           
        if warp_matrix[i,0,2]<warp_matrix_max[0]:
            warp_matrix_max[0]=warp_matrix[i,0,2]
            
        if warp_matrix[i,0,2]>warp_matrix_max[1]:
            warp_matrix_max[1]=warp_matrix[i,0,2]

        if warp_matrix[i,1,2]<warp_matrix_max[2]:
            warp_matrix_max[2]=warp_matrix[i,1,2]
            
        if warp_matrix[i,1,2]>warp_matrix_max[3]:
            warp_matrix_max[3]=warp_matrix[i,1,2]
            
        print(str(i) +' / '+str(hs_stack.data.shape[0]-1))
 
    hs_stack.crop_image(bottom=hs_stack.data.shape[2]-np.int(warp_matrix_max[3]), \
                        top = -np.int(warp_matrix_max[2]), \
                        left=-np.int(warp_matrix_max[0]), \
                        right = hs_stack.data.shape[1]-np.int(warp_matrix_max[1]))
       
    return

In [ ]:
TEM_bin4 = hs.load('16)_380K_bin4_ali.dm4')

In [ ]:
TEM_bin4.crop_image(top=80., left=20., bottom=115., right=100.)

In [ ]:
warp_matrix_returned_bin4 = align2D_cv(TEM_bin4, cv2.MOTION_TRANSLATION)

In [ ]:
warp_matrix_returned_bin4 = np.load('warp_matrix_returned_bin4.npy')

np.save('warp_matrix_returned_bin4.npy',warp_matrix_returned_bin4)

In [ ]:
TEM_bin4 = hs.load('16)_380K_bin4_ali.dm4')

In [ ]:
align2D_cv_only_align(TEM_bin4, warp_matrix_returned_bin4, cv2.MOTION_TRANSLATION)

In [ ]:
TEM_bin4.plot()

In [ ]:
warp_matrix_returned_bin2 = np.empty_like(warp_matrix_returned_bin4)
warp_matrix_returned_bin2 = warp_matrix_returned_bin4
warp_matrix_returned_bin2[:,0,2] = 2*warp_matrix_returned_bin4[:,0,2]
warp_matrix_returned_bin2[:,1,2] = 2*warp_matrix_returned_bin4[:,1,2]

In [ ]:
TEM_bin2 = hs.load('16)_380K_bin2_ali.dm4')

align2D_cv_only_align(TEM_bin2, warp_matrix_returned_bin2, cv2.MOTION_TRANSLATION)

In [ ]:
TEM_bin2.plot()

In [ ]:
TEM_bin2.save('TEM_bin2_aligned.hdf5')

In [ ]:
TEM_bin2.plot()

# Not binned

In [ ]:
TEM = hs.load('16)_380K_crop.hdf5')

In [ ]:
warp_matrix_returned = align2D_cv(TEM, cv2.MOTION_TRANSLATION)

In [ ]:
np.save('warp_matrix_returned.npy', warp_matrix_returned)

In [ ]:
TEM.plot()

### FFT videos

In [ ]:
TEM_croptr = hs.load('Extract_16)_380K_aligned_bin2_tr.dm4')

In [ ]:
TEM_croptr.plot()

In [ ]:
TEM_croptr_fft = TEM_croptr.fft(shift=True, apodization=False)

In [ ]:
TEM_croptr_fft.data = np.log(np.abs(TEM_croptr_fft.data))

In [ ]:
TEM_cropbl = hs.load('Extract_16)_380K_aligned_bin2_bl.dm4')

In [ ]:
TEM_cropbl.plot()

In [ ]:
TEM_cropbl_fft = TEM_cropbl.fft(shift=True, apodization=False)

In [ ]:
TEM_cropbl_fft.data = np.log(np.abs(TEM_cropbl_fft.data))

### Save images

In [ ]:
TEM2 = hs.load('16)_380K_aligned_bin2.hdf5')

img_nbr=0
grid = GridSpec(nrows=2, ncols=4, left=0.1, bottom=0.235, right=0.9, top=0.765, wspace=0.05, hspace=0.05)

fig = plt.figure(0)
fig.clf()
rect1 = patches.Rectangle((912,149),512,512,linewidth=4,edgecolor='orange',facecolor='none', linestyle = '--')
rect2 = patches.Rectangle((227,920),256,256,linewidth=4,edgecolor='purple',facecolor='none', linestyle = '--')

ax1 = fig.add_subplot(grid[0:2, 0:2])
plt.setp(ax1, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
im = ax1.imshow(TEM2.data[img_nbr], cmap='gray', vmin=0, vmax=2.7*np.mean(TEM2.data[img_nbr]))
ax1.add_patch(rect1)
ax1.add_patch(rect2)

ax20 = fig.add_subplot(grid[0, 2])
plt.setp(ax20, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax20.spines[pos].set_edgecolor('orange')
im = ax20.imshow(TEM_croptr.data[img_nbr], cmap='gray', \
                vmin=0, \
                vmax=2.7*np.mean(TEM_croptr.data[img_nbr]))

ax21 = fig.add_subplot(grid[0, 3])
plt.setp(ax21, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax21.spines[pos].set_edgecolor('orange')
im = ax21.imshow(TEM_croptr_fft.data[img_nbr], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[img_nbr]), \
                vmax=1.3*np.mean(TEM_croptr_fft.data[img_nbr]))
ax21.arrow(354, 46, -17.5, 38, head_width=20., color='orange')
  
ax30 = fig.add_subplot(grid[1, 2])
plt.setp(ax30, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax30.spines[pos].set_edgecolor('yellow')
im = ax30.imshow(TEM_cropbl.data[img_nbr], cmap='gray', \
                vmin=0, \
                vmax=2.7*np.mean(TEM_cropbl.data[img_nbr]))
    
ax31 = fig.add_subplot(grid[1, 3])
plt.setp(ax31, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax31.spines[pos].set_edgecolor('yellow')
im = ax31.imshow(TEM_cropbl_fft.data[img_nbr], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[img_nbr]), \
                vmax=1.3*np.mean(TEM_cropbl_fft.data[img_nbr]))
ax31.arrow(129, 47, -17.5/2, 38/2, head_width=20./2, color='purple')
ax31.arrow(108, 23, -17.5/2, 38/2, head_width=20./2, color='orange')

fig.savefig('TEM_exp16_nbr'+str(img_nbr)+'.png', dpi=600)

#### IMG+FFT

In [ ]:
img_nbr=107

#fig = plt.figure(0)
fig, axes = plt.subplots(nrows=2, ncols=3, sharex=False, sharey=False, squeeze=True, \
                         figsize=(8,6))#, gridspec_kw={'width_ratios': [1.2, 1]})
grid = GridSpec(nrows=2, ncols=3, left=0.125, bottom=0.15, right=0.88, top=0.85, wspace=0.05, hspace=0.05)

fig.clf()
rect1 = patches.Rectangle((912,149),512,512,linewidth=4,edgecolor='orange',facecolor='none', linestyle = '--')
rect2 = patches.Rectangle((227,920),256,256,linewidth=4,edgecolor='purple',facecolor='none', linestyle = '--')

ax1 = fig.add_subplot(grid[0:2, 0:2])
plt.setp(ax1, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
im = ax1.imshow(TEM2.data[img_nbr], cmap='gray', vmin=0, vmax=2.7*np.mean(TEM2.data[img_nbr]))
ax1.add_patch(rect1)
ax1.add_patch(rect2)

ax21 = fig.add_subplot(grid[0, 2])
plt.setp(ax21, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax21.spines[pos].set_edgecolor('orange')
im = ax21.imshow(TEM_croptr_fft.data[img_nbr], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[img_nbr]), \
                vmax=1.3*np.mean(TEM_croptr_fft.data[img_nbr]))
ax21.arrow(354, 46, -17.5, 38, head_width=20., color='orange')
    
ax31 = fig.add_subplot(grid[1, 2])
plt.setp(ax31, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax31.spines[pos].set_edgecolor('purple')
im = ax31.imshow(TEM_cropbl_fft.data[img_nbr], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[img_nbr]), \
                vmax=1.3*np.mean(TEM_cropbl_fft.data[img_nbr]))
ax31.arrow(129, 47, -17.5/2, 38/2, head_width=20./2, color='purple')
ax31.arrow(108, 23, -17.5/2, 38/2, head_width=20./2, color='orange')

fig.savefig('TEM_exp16_nbr_small_'+str(img_nbr)+'.png', dpi=600)

#### IMG no FFT

In [ ]:
img_nbr=19

#fig = plt.figure(0)
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, squeeze=True, \
                         figsize=(8,6), gridspec_kw={'width_ratios': [5, 1]})
grid = GridSpec(nrows=1, ncols=2, left=0.1, bottom=0.25, right=0.9, top=0.75, wspace=0.01, hspace=0.01)

fig.clf()
rect2 = patches.Rectangle((227,920),256,256,linewidth=4,edgecolor='purple',facecolor='none', linestyle = '--')

ax1 = fig.add_subplot(grid[0, 0])
plt.setp(ax1, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
im = ax1.imshow(TEM2.data[img_nbr], cmap='gray', vmin=0, vmax=2.7*np.mean(TEM2.data[img_nbr]))
ax1.add_patch(rect2)
   
ax31 = fig.add_subplot(grid[0, 1])
plt.setp(ax31, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax31.spines[pos].set_edgecolor('purple')
im = ax31.imshow(TEM_cropbl.data[img_nbr], cmap='gray', \
                vmin=0.4*np.mean(TEM_cropbl.data[img_nbr]), \
                vmax=2*np.mean(TEM_cropbl.data[img_nbr]))

fig.savefig('TEM_exp16_bl_'+str(img_nbr)+'.png', dpi=600)

### Video FFT+graph

### Video

In [ ]:
FFMpegWriter = animation.writers['ffmpeg']
metadata = dict(title='Movie Test', artist='Matplotlib', comment='Movie support!')
writer = FFMpegWriter(fps=5, metadata=metadata)

In [ ]:
mpl.rcParams['axes.linewidth'] = 3.5 #set the value globally

grid = GridSpec(nrows=3, ncols=3, left=0.125, bottom=0.1, right=0.9, top=0.95, wspace=0.05, hspace=0.1)
fig = plt.figure(figsize=(6,6))
fig.clf()

rect1 = patches.Rectangle((912,149),512,512,linewidth=4,edgecolor='orange',facecolor='none', linestyle = '--')
rect2 = patches.Rectangle((227,920),256,256,linewidth=4,edgecolor='yellow',facecolor='none', linestyle = '--')

ax1 = fig.add_subplot(grid[0:2, 0:2])
plt.setp(ax1, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
im = ax1.imshow(TEM2.data[img_nbr], cmap='gray', vmin=0, vmax=2.7*np.mean(TEM2.data[img_nbr]))
ax1.add_patch(rect1)
ax1.add_patch(rect2)

ax21 = fig.add_subplot(grid[0, 2])
plt.setp(ax21, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax21.spines[pos].set_edgecolor('orange')
im = ax21.imshow(TEM_croptr_fft.data[img_nbr], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[img_nbr]), \
                vmax=1.3*np.mean(TEM_croptr_fft.data[img_nbr]))
ax21.arrow(354, 46, -17.5, 38, head_width=20., color='orange')
    
ax31 = fig.add_subplot(grid[1, 2])
plt.setp(ax31, xticks=[], xticklabels=[], yticks=[], yticklabels=[])
for pos in ['top', 'bottom', 'right', 'left']:
    ax31.spines[pos].set_edgecolor('yellow')
im = ax31.imshow(TEM_cropbl_fft.data[img_nbr], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[img_nbr]), \
                vmax=1.3*np.mean(TEM_cropbl_fft.data[img_nbr]))
ax31.arrow(129, 47, -17.5/2, 38/2, head_width=20./2, color='yellow')
ax31.arrow(108, 23, -17.5/2, 38/2, head_width=20./2, color='orange')

# x axis graph
ax4 = fig.add_subplot(grid[2:3, 0:3])
ax4.set_xlabel('Time [min]',fontsize=10)
ax5 = ax4.twinx()  # instantiate a second axes that shares the same x-axis

# Voltage graph
#ax4.plot(Tag_TEM_image[:,0]/60,Tag_TEM_image[:,1], color='red')
#ax4.plot(Tag_TEM_image[:,0]/60, Tag_TEM_image[:,4], lw=5, color='red')
ax4.set_ylabel('Voltage [mV]',fontsize=10)
ax4.set_xlim(0, TEM2.data.shape[0]*20/60)
ax4.set_ylim(np.min(Tag_TEM_image[:,1])*0.8, np.max(Tag_TEM_image[:,1])*1.2)

# O2/H2 current graph
#ax5.plot(Tag_TEM_image[:,0]/60-4,100*(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
#         - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, color='black')
#ax5.plot(Tag_TEM_image[:,0]/60,100*(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
#        - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, dashes=[6, 2], color='black')
ax5.set_ylabel('O$_2$/H$_2$ current ratio (x100)',fontsize=10)

ax5.set_ylim(100*(np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
                - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] ))), \
                100*(np.max(Tag_TEM_image[:,2]/Tag_TEM_image[:,5] \
                - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5]))*1.1))

plt.setp(ax4.get_xticklabels(), fontsize=7)
plt.setp(ax4.get_yticklabels(), fontsize=7)
plt.setp(ax5.get_yticklabels(), fontsize=7)
ax5.yaxis.set_ticks(np.arange(0, 4.5, 1))

plt.plot()

In [ ]:
with writer.saving(fig, "Day2_exp16_FFT.mp4", dpi=300):
    for i in range(1,TEM2.data.shape[0],1):
        im = ax1.imshow(TEM2.data[i], cmap='gray', vmin=0, vmax=2.7*np.mean(TEM2.data[img_nbr]))
        im = ax21.imshow(TEM_croptr_fft.data[i], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[i]), \
                vmax=1.3*np.mean(TEM_croptr_fft.data[i]))
        im = ax31.imshow(TEM_cropbl_fft.data[i], cmap='gray', \
                vmin=np.mean(TEM_croptr_fft.data[i]), \
                vmax=1.3*np.mean(TEM_cropbl_fft.data[i]))
        im = ax4.plot(Tag_TEM_image[:i,0]/60,Tag_TEM_image[:i,1], color='red')
        im = ax4.plot(Tag_TEM_image[:i,0]/60, Tag_TEM_image[:i,4], lw=5, color='red')
        im = ax5.plot(Tag_TEM_image[:i,0]/60-4,100*(Tag_TEM_image[:i,2]/Tag_TEM_image[:i,5] \
                 - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, color='black')
        im = ax5.plot(Tag_TEM_image[:i,0]/60,100*(Tag_TEM_image[:i,2]/Tag_TEM_image[:i,5] \
                - np.min(Tag_TEM_image[:,2]/Tag_TEM_image[:,5])), lw=5, dashes=[6, 2], color='black')
        
        textstr = '\n'.join((
            'Img nbr: ' + str(i),
            'Time: ' + str(round(20*Tag_TEM_image[(i),3]/60,2)) +' min',
            str(round(Tag_TEM_image[(i),1],2)) + '  V'))
       
        textbox = plt.text(20*40./60, 1, textstr, fontsize=7, verticalalignment='top', \
                           backgroundcolor='white')   
        writer.grab_frame()
        textbox.remove()
        print(str(i) + ' over ' + str(TEM2.data.shape[0]-1))